# Stream Ciphers

In [154]:
from typing import List, Tuple

In [137]:
def sum32(a: int, b: int) -> int:
    # Suma modulo 2^32
    return (a + b) & ((1 << 32) - 1)

assert(sum32(2**32, 3) == 3), "Error"

In [138]:
def text_to_int(string: str) -> int:
    # I copy this from StackOverFlow
    nchars = len(string)
    return sum(ord(string[byte])<<8*(nchars-byte-1) for byte in range(nchars))

assert(text_to_int("expa"[::-1]) == 0x61707865), "Error"

In [152]:
# I dont remmber where I found this
def int_to_text(integer: int) -> str:
    result = ""
    while integer > 0:
        byte = integer & 0xFF
        result = chr(byte) + result
        integer >>= 8
    
    return result

assert("expa" == int_to_text(0x61707865)[::-1]), "Error"

In [140]:
# Chatgpt assited generated code 
# Do no ask me how it works
def to_words(value: int, word_size: int, words_num: int) -> List[int]:
    hex_string = value[2:]  # Remove '0x' prefix
    binary_string = bin(int(hex_string, 16))[2:].zfill(len(hex_string) * 4)  # Convert to binary
    padding_bits = word_size * words_num - len(binary_string)
    if padding_bits < 0:
        raise ValueError("Number of words specified is insufficient for the given value.")
    binary_string = binary_string.zfill(word_size * words_num)  # Add leading zeros for padding
    words = [int(binary_string[i:i+word_size], 2) for i in range(0, len(binary_string), word_size)]
    return words

assert(to_words(hex(0x112233445566778899aabbccddeeff00), 32, 8) == [0x0, 0x0, 0x0, 0x0, 0x11223344, 0x55667788, 0x99aabbcc, 0xddeeff00]), "Error"

In [167]:
def from_words(words: List[int], word_size: int) -> int:
    binary_string = ''.join(format(word, f'0{word_size}b') for word in words)
    hex_string = hex(int(binary_string, 2))
    return hex_string

assert(hex(0x112233445566778899aabbccddeeff00) == from_words([0x0, 0x0, 0x0, 0x0, 0x11223344, 0x55667788, 0x99aabbcc, 0xddeeff00], 32)), "Error"


# Salsa

### Pad

In [151]:
# constant word (32 bits each)
# "Expand 32-byte k"
c = [text_to_int("expa"[::-1]),
     text_to_int("nd 3"[::-1]),
     text_to_int("2-by"[::-1]),
     text_to_int("te k"[::-1])]

def pad(s: int, j: int, n: int) -> int:
    """A padding function that combies a 256 bits seed s with a 64 bit counter j (j0, j1 of 32 bits) and a b4 bit nonce (n0, n1 of 32 bits) to form a 512 bit block denoted x0, ..., x15 of 32 bits
    
    Args:
        s: Seed (256 bits, 8 words of 32 bits)
        j: counter (64 bits)
        n: Nonce (64 bits)

    Returns:
        512 bits (16 words 32 bits)
    """

    # Convert to words
    s = to_words(hex(s), 32, 8)
    j = to_words(hex(j), 32, 2)
    n = to_words(hex(n), 32, 2)

    return [
        c[0], s[0], s[1], s[2],
        s[3], c[1], n[0], n[1],
        j[0], j[1], c[2], s[4],
        s[5], s[6], s[7], c[3]
    ]

assert(
    pad(0x112233445566778899aabbccddeeff00, 0x0123456789abcdef, 0xfedcba9876543210) == [0x61707865, 0x0, 0x0, 0x0, 0x0, 0x3320646e, 0xfedcba98, 0x76543210, 0x1234567, 0x89abcdef, 0x79622d32, 0x11223344, 0x55667788, 0x99aabbcc, 0xddeeff00, 0x6b206574]
), "Error"

### Permutation $\pi$

Interating a simple permutation a fized number of times

In [153]:
def rot(w: int, r: int) -> int: 
    """Rotate lest for 32 bits

    Args:
        w: word to rotate
        r: I dont remember
    """

    mask = 0xffffffff
    return ((w << r) & mask) | (w >> (32 - r)) 

In [165]:
def QR_salsa(a: int, b: int, c: int, d: int) -> Tuple[int, int, int, int]: 
    # Quater round
    b = b ^ rot(sum32(a, d), 7)
    c = c ^ rot(sum32(b, a), 9)
    d = d ^ rot(sum32(c, b), 13)
    a = a ^ rot(sum32(d, c), 18)
    
    return a, b, c, d

In [169]:
ROUNDS = 20

def perm_salsa(x: int): 
    """Permutation 

    Args:
        x: 512 bits (16 words 32 bits)

    Returns:
        512 bits (16 words 32 bits)
    """
    
    i = x.copy()
    
    for _ in range(0, ROUNDS, 2): 
        # Odd round
        i[0], i[4], i[8], i[12] = QR_salsa(i[0], i[4], i[8], i[12]) # Col1
        i[5], i[9], i[13], i[1] = QR_salsa(i[5], i[9], i[13], i[1]) # Col2
        i[10], i[14], i[2], i[6] = QR_salsa(i[10], i[14], i[2], i[6]) # Col3
        i[15], i[3], i[7], i[11] = QR_salsa(i[15], i[3], i[7], i[11]) # Col4
        # Even round 
        i[0], i[1], i[2], i[3] = QR_salsa(i[0], i[1], i[2], i[3]) # Row1
        i[5], i[6], i[7], i[4] = QR_salsa(i[5], i[6], i[7], i[4]) # Row2
        i[10], i[11], i[8], i[9] = QR_salsa(i[10], i[11], i[8], i[9]) # Row3
        i[15], i[12], i[13], i[14] = QR_salsa(i[15], i[12], i[13], i[14]) # Row 4
    return i

### Generador psudoaletorio $G$

In [176]:
s = text_to_int("*Thirty-two byte (256 bits) key*")

initial_state = pad(s, 0, 0)
pi = perm_salsa(initial_state)
len(bin(int(from_words(perm_salsa(pi), 32), 16)))

514

### Example 

In [ ]:
s = text_to_int("*Thirty-two byte (256 bits) key*")
from_words(perm_salsa(s), 32)

## ChaCha

In [51]:
def QR_chacha(a, b, c, d):
    a = sum(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum(a, b)
    
    return a, b, c, d